<a href="https://colab.research.google.com/github/ScriptSherpa/AI-and-Ml-journey/blob/main/Apriori_The_Bread_Basket%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Reading and Preprocessing

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations


In [4]:
## Specify the input file here
#File needs to be a **csv** of the following format:

###
'''
item1, item2, item3, ... so on
 , t, ...
t, t, t,...
t, t, ...
... so on...'''

'\nitem1, item2, item3, ... so on\n , t, ...\nt, t, t,...\nt, t, ...\n... so on...'

In [6]:
df = pd.read_csv("/content/test1.csv", low_memory=False)


In [7]:
df.head()

,handphone,laptop,charger,powerbank,tablet
0,t,t,NaN,NaN,NaN
1,t,t,t,NaN,NaN
2,t,t,t,t,NaN
3,t,t,NaN,NaN,t
4,t,NaN,t,NaN,t


Indexing each item from the header of the data file.

In [8]:
item_list = list(df.columns)
item_dict = dict()

In [9]:
for i, item in enumerate(item_list):
    item_dict[item] = i + 1


In [10]:
item_list = list(df.columns)
item_dict = dict()

for i, item in enumerate(item_list):
    item_dict[item] = i + 1

item_dict

{'handphone': 1, 'laptop': 2, 'charger': 3, 'powerbank': 4, 'tablet': 5}

Extracting the transactions from the data.


In [12]:
transactions = list()

for i, row in df.iterrows():
    transaction = set()

    for item in item_dict:
        if row[item] == 't':
            transaction.add(item_dict[item])
    transactions.append(transaction)


In [ ]:
## Utility Functions


In [ ]:
##**get_support** function evaluates the support value for a set given all the transactions.


In [13]:
def get_support(transactions, item_set):
    match_count = 0
    for transaction in transactions:
        if item_set.issubset(transaction):
            match_count += 1

    return float(match_count/len(transactions))

In [14]:
def self_join(frequent_item_sets_per_level, level):
    current_level_candidates = list()
    last_level_items = frequent_item_sets_per_level[level - 1]

    if len(last_level_items) == 0:
        return current_level_candidates

    for i in range(len(last_level_items)):
        for j in range(i+1, len(last_level_items)):
            itemset_i = last_level_items[i][0]
            itemset_j = last_level_items[j][0]
            union_set = itemset_i.union(itemset_j)

            if union_set not in current_level_candidates and len(union_set) == level:
                current_level_candidates.append(union_set)

    return current_level_candidates


In [15]:
def get_single_drop_subsets(item_set):
    single_drop_subsets = list()
    for item in item_set:
        temp = item_set.copy()
        temp.remove(item)
        single_drop_subsets.append(temp)

    return single_drop_subsets

def is_valid_set(item_set, prev_level_sets):
    single_drop_subsets = get_single_drop_subsets(item_set)

    for single_drop_set in single_drop_subsets:
        if single_drop_set not in prev_level_sets:
            return False
    return True

def pruning(frequent_item_sets_per_level, level, candidate_set):
    post_pruning_set = list()
    if len(candidate_set) == 0:
        return post_pruning_set

    prev_level_sets = list()
    for item_set, _ in frequent_item_sets_per_level[level - 1]:
        prev_level_sets.append(item_set)

    for item_set in candidate_set:
        if is_valid_set(item_set, prev_level_sets):
            post_pruning_set.append(item_set)

    return post_pruning_set

In [16]:
## Apriori Algorithm


In [17]:
from collections import defaultdict

def apriori(min_support):
    frequent_item_sets_per_level = defaultdict(list)
    print("level : 1", end = " ")

    for item in range(1, len(item_list) + 1):
        support = get_support(transactions, {item})
        if support >= min_support:
            frequent_item_sets_per_level[1].append(({item}, support))

    for level in range(2, len(item_list) + 1):
        print(level, end = " ")
        current_level_candidates = self_join(frequent_item_sets_per_level, level)

        post_pruning_candidates = pruning(frequent_item_sets_per_level, level, current_level_candidates)
        if len(post_pruning_candidates) == 0:
            break

        for item_set in post_pruning_candidates:
            support = get_support(transactions, item_set)
            if support >= min_support:
                frequent_item_sets_per_level[level].append((item_set, support))

    return frequent_item_sets_per_level

In [ ]:
### Specify the **minimum support** value here


In [18]:
min_support = 0.005
frequent_item_sets_per_level = apriori(min_support)

level : 1 2 3 4 5 

In [19]:

for level in frequent_item_sets_per_level:
    print(len(frequent_item_sets_per_level[level]))

5
10
7
2


In [20]:
for level in frequent_item_sets_per_level:
    print(frequent_item_sets_per_level[level])

[({1}, 0.8), ({2}, 0.6), ({3}, 0.6), ({4}, 0.4), ({5}, 0.4)]
[({1, 2}, 0.5), ({1, 3}, 0.5), ({1, 4}, 0.2), ({1, 5}, 0.3), ({2, 3}, 0.4), ({2, 4}, 0.2), ({2, 5}, 0.2), ({3, 4}, 0.2), ({3, 5}, 0.2), ({4, 5}, 0.1)]
[({1, 2, 3}, 0.3), ({1, 2, 4}, 0.1), ({1, 2, 5}, 0.2), ({1, 3, 4}, 0.1), ({1, 3, 5}, 0.2), ({2, 3, 4}, 0.2), ({2, 3, 5}, 0.1)]
[({1, 2, 3, 4}, 0.1), ({1, 2, 3, 5}, 0.1)]


In [ ]:
## Generating Association Rules

#Prepare input for calculating association rules: Create a dictionary of each frequent itemset against its support value.

In [21]:
item_support_dict = dict()
item_list = list()

key_list = list(item_dict.keys())
val_list = list(item_dict.values())

for level in frequent_item_sets_per_level:
    for set_support_pair in frequent_item_sets_per_level[level]:
        for i in set_support_pair[0]:
            item_list.append(key_list[val_list.index(i)])
        item_support_dict[frozenset(item_list)] = set_support_pair[1]
        item_list = list()


In [22]:
item_support_dict


{frozenset({'handphone'}): 0.8,
 frozenset({'laptop'}): 0.6,
 frozenset({'charger'}): 0.6,
 frozenset({'powerbank'}): 0.4,
 frozenset({'tablet'}): 0.4,
 frozenset({'handphone', 'laptop'}): 0.5,
 frozenset({'charger', 'handphone'}): 0.5,
 frozenset({'handphone', 'powerbank'}): 0.2,
 frozenset({'handphone', 'tablet'}): 0.3,
 frozenset({'charger', 'laptop'}): 0.4,
 frozenset({'laptop', 'powerbank'}): 0.2,
 frozenset({'laptop', 'tablet'}): 0.2,
 frozenset({'charger', 'powerbank'}): 0.2,
 frozenset({'charger', 'tablet'}): 0.2,
 frozenset({'powerbank', 'tablet'}): 0.1,
 frozenset({'charger', 'handphone', 'laptop'}): 0.3,
 frozenset({'handphone', 'laptop', 'powerbank'}): 0.1,
 frozenset({'handphone', 'laptop', 'tablet'}): 0.2,
 frozenset({'charger', 'handphone', 'powerbank'}): 0.1,
 frozenset({'charger', 'handphone', 'tablet'}): 0.2,
 frozenset({'charger', 'laptop', 'powerbank'}): 0.2,
 frozenset({'charger', 'laptop', 'tablet'}): 0.1,
 frozenset({'charger', 'handphone', 'laptop', 'powerbank'}

### Utility Function

**find_subset** finds all the subsets of the given itemset.

In [23]:
def find_subset(item, item_length):
    combs = []
    for i in range(1, item_length + 1):
        combs.append(list(combinations(item, i)))

    subsets = []
    for comb in combs:
        for elt in comb:
            subsets.append(elt)

    return subsets

**association_rules** generates the association rules in accordance with the given *minimum confidence* value and the provided dictionary of itemsets against their support values. For itemsets of more than one element, it first finds all their subsets. For every subset A, it calculates the set B = itemset-A. If B is not empty, the confidence of B is calculated. If this value is more than *minimum confidence* value, the rule *A->B* is added to the list.


In [24]:
def association_rules(min_confidence, support_dict):
    rules = list()
    for item, support in support_dict.items():
        item_length = len(item)

        if item_length > 1:
            subsets = find_subset(item, item_length)

            for A in subsets:
                B = item.difference(A)

                if B:
                    A = frozenset(A)

                    AB = A | B

                    confidence = support_dict[AB] / support_dict[A]
                    if confidence >= min_confidence:
                        rules.append((A, B, confidence))

    return rules

In [ ]:
### Specify Minimum confidence value here


In [ ]:
association_rules = association_rules(min_confidence = 0.6, support_dict = item_support_dict)


In [ ]:
### Printing the output in the required format


In [26]:
### Specify Minimum confidence value here

min_confidence = 0.6
rules = association_rules(min_confidence = min_confidence, support_dict = item_support_dict) #assign output of association_rules function to variable rules

### Printing the output in the required format

print("Number of rules: ", len(rules), "\n") #use variable rules here instead of association_rules

for rule in rules: #use variable rules here instead of association_rules
    print('{0} -> {1} <confidence: {2}>'.format(set(rule[0]), set(rule[1]), rule[2]))

Number of rules:  19 

{'handphone'} -> {'laptop'} <confidence: 0.625>
{'laptop'} -> {'handphone'} <confidence: 0.8333333333333334>
{'handphone'} -> {'charger'} <confidence: 0.625>
{'charger'} -> {'handphone'} <confidence: 0.8333333333333334>
{'tablet'} -> {'handphone'} <confidence: 0.7499999999999999>
{'charger'} -> {'laptop'} <confidence: 0.6666666666666667>
{'laptop'} -> {'charger'} <confidence: 0.6666666666666667>
{'handphone', 'charger'} -> {'laptop'} <confidence: 0.6>
{'handphone', 'laptop'} -> {'charger'} <confidence: 0.6>
{'charger', 'laptop'} -> {'handphone'} <confidence: 0.7499999999999999>
{'handphone', 'tablet'} -> {'laptop'} <confidence: 0.6666666666666667>
{'laptop', 'tablet'} -> {'handphone'} <confidence: 1.0>
{'handphone', 'tablet'} -> {'charger'} <confidence: 0.6666666666666667>
{'charger', 'tablet'} -> {'handphone'} <confidence: 1.0>
{'powerbank', 'charger'} -> {'laptop'} <confidence: 1.0>
{'powerbank', 'laptop'} -> {'charger'} <confidence: 1.0>
{'handphone', 'charger